# This notebook exemplifies the calculation of the horizontal and vertical derivatives by using the *classical equivalent layer* approach.

In [ ]:
# Import all python modules
import numpy
from matplotlib import pyplot

In [ ]:
from codes.auxiliars import regional

In [ ]:
# Import some developed modules
from codes.prism import prism_gz
from codes.sphere import sphere_gz
from codes.grids import regular_grid
from codes.equivalentlayer import layer, mat_grav_gz

In [ ]:
# Statistical module
from codes.auxiliars import addnoise
from codes.statistical import analysis

In [ ]:
# Data shape:
nx = 30
ny = 40
shape = (nx, ny)

In [ ]:
# Number of observations:
xo, yo, zo = regular_grid((-2000., 2000., -2000., 2000.), shape, -200.)

In [ ]:
# Synthetic data produced by a vertical dike model (vertical attraction):
# (xmin, xmax, ymin, ymax, zmin, zmax, rho(km/m3) )
dike1 = (-300., 150., -100., 280., 100., 750., 2800.0)
#dike2 = (1000., 1350., 1400., 1680., 90., 1950., 2900.0)
gz = prism_gz(xo, yo, zo, dike1) #+ prism_gz(xo, yo, zo, dike2)

In [ ]:
print 'Analysing gz data:'
_ = analysis(gz)

In [ ]:
# Adding noise
gz = addnoise(gz, 1., 0.1)

In [ ]:
print 'Analysing noised gz data:'
_ = analysis(gz)

#### Creating the equivalent layer:

In [ ]:
# Define a mesh of masses point with unit volume:
# 1) horizontal coordinates:
area = (-2000., 2000., -2000., 2000.) # meters
shape_layer = (10, 10)
# 2) define the depth of the layer and the number of point in area:
eqlayer = layer(area, shape_layer, 700.)

In [ ]:
# computing the Jacobian matrix:
A = mat_grav_gz(xo, yo, zo, eqlayer)

In [ ]:
# dimensions of the sensitivity matrix of the eqlayer problem:
M = shape_layer[0] * shape_layer[1]
N = shape[0] * shape[1]
print 'Number of observations:', N
print 'Number of depth sources:', M

### Solve the least-square problem for the two possible cases:

In [ ]:
# Estimation of vector parameters:
if N >= M: #overdetermined case
    mu = 0.0 #1.0e-3
    I = numpy.identity(M)
    trace = numpy.trace(A.T.dot(A))/M
    vec = numpy.linalg.solve(numpy.dot(A.T, A) + mu*trace*I, numpy.dot(A.T,gz))
else:# underterminated case
    mu = 1.e-2
    I = numpy.identity(N)
    trace = numpy.trace(A.T.dot(A))/N
    aux = numpy.linalg.solve(numpy.dot(A, A.T) + mu*trace*I, gz)
    vec = numpy.dot(A.T, aux)

In [ ]:
# Computing the predicted data
gz_predicted = numpy.dot(A, vec)

In [ ]:
print 'Original gz_data:'
_ = analysis(gz)
print 'Predicted gz data:'
_ = analysis(gz_predicted)

In [ ]:
from codes.auxiliars import residual

In [ ]:
res, norm, mean, deviation = residual(gz, gz_predicted)

In [ ]:
pyplot.figure(figsize=(16, 4))

pyplot.subplot(1, 3, 1)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                gz.reshape(nx,ny), 20, cmap = pyplot.cm.jet)
pyplot.title('Observed gz (mGal)')
pyplot.xlabel('X coordinate (m)')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()

pyplot.subplot(1, 3, 2)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                gz_predicted.reshape(nx,ny), 20, cmap = pyplot.cm.jet)
pyplot.title('Predicted gz by eqlayer (mGal)')
pyplot.xlabel('X coordinate (m)')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()

pyplot.subplot(1, 3, 3)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                res.reshape(nx,ny), 10, cmap = pyplot.cm.jet)
pyplot.title('Calculated residual (mGal)')
pyplot.xlabel('X coordinate (m)')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()

pyplot.show()

# Test for tensor elements

In [ ]:
# Variation along the directinos
delta = 10.

**Calculating the numerical derivative**

In [ ]:
# In X
xi = prism_gz(xo - delta, yo, zo, dike1) + prism_gz(xo - delta, yo, zo, dike2)
xf = prism_gz(xo + delta, yo, zo, dike1) + prism_gz(xo + delta, yo, zo, dike2)
diffx = (xf - xi)/(2.*delta)

In [ ]:
# In Y
yi = prism_gz(xo, yo - delta, zo, dike1) + prism_gz(xo, yo - delta, zo, dike2)
yf = prism_gz(xo, yo + delta, zo, dike1) + prism_gz(xo, yo + delta, zo, dike2)
diffy = (yf - yi)/(2.*delta)


In [ ]:
# In Z
zi = prism_gz(xo, yo, zo - 0.1*delta, dike1) + prism_gz(xo, yo, zo - 0.1*delta, dike2)
zf = prism_gz(xo, yo, zo + 0.1*delta, dike1) + prism_gz(xo, yo, zo + 0.1*delta, dike2)
diffz = (zf - zi)/(2.*0.1*delta)


In [ ]:
# Import the transformation matrix for derivatives
from codes.equivalentlayer import layer, mat_grav_gz, mat_grav_gz_xyz

In [ ]:
gzx, gzy, gzz = mat_grav_gz_xyz(xo, yo, zo, eqlayer)

In [ ]:
# Computing the tensor elements
gzx_predicted = numpy.dot(gzx, vec)
gzy_predicted = numpy.dot(gzy, vec)
gzz_predicted = numpy.dot(gzz, vec)

In [ ]:
reszx, _, _, _ = residual(diffx, gzx_predicted)
reszy, _, _, _ = residual(diffy, gzy_predicted)
reszz, _, _, _ = residual(diffz, gzz_predicted)

In [ ]:
pyplot.figure(figsize=(24, 12))

pyplot.subplot(3, 3, 1)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                diffx.reshape(nx,ny), 20, cmap = pyplot.cm.jet)
pyplot.title('Numerical x-derivative of gz')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()

pyplot.subplot(3, 3, 2)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                diffy.reshape(nx,ny), 20, cmap = pyplot.cm.jet)
pyplot.title('Numerical y-derivative of gz')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()

pyplot.subplot(3, 3, 3)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                diffz.reshape(nx,ny), 20, cmap = pyplot.cm.jet)
pyplot.title('Numerical z-derivative of gz')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()

pyplot.subplot(3, 3, 4)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                gzx_predicted.reshape(nx,ny), 20, cmap = pyplot.cm.jet)
pyplot.title('X-derivative by equivalent layer')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()

pyplot.subplot(3, 3, 5)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                gzy_predicted.reshape(nx,ny), 20, cmap = pyplot.cm.jet)
pyplot.title('Y-derivative by equivalent layer')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()

pyplot.subplot(3, 3, 6)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                gzz_predicted.reshape(nx,ny), 20, cmap = pyplot.cm.jet)
pyplot.title('Z-derivative by equivalent layer')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()

pyplot.subplot(3, 3, 7)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                reszx.reshape(nx,ny), 10, cmap = pyplot.cm.jet)
pyplot.title('Calculated residual (mGal)')
pyplot.xlabel('X coordinate (m)')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()

pyplot.subplot(3, 3, 8)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                reszy.reshape(nx,ny), 10, cmap = pyplot.cm.jet)
pyplot.title('Calculated residual (mGal)')
pyplot.xlabel('X coordinate (m)')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()

pyplot.subplot(3, 3, 9)
pyplot.contourf(xo.reshape(nx,ny), yo.reshape(nx,ny), 
                reszz.reshape(nx,ny), 10, cmap = pyplot.cm.jet)
pyplot.title('Calculated residual (mGal)')
pyplot.xlabel('X coordinate (m)')
pyplot.ylabel('Y coordinate (m)')
pyplot.xticks(numpy.linspace(xo.min(), xo.max(), 5))
pyplot.yticks(numpy.linspace(yo.min(), yo.max(), 5))
pyplot.colorbar()


pyplot.show()